# Requirements

In [1]:
import os, sys

pkg_path = 'D:\\Projects\\System_Trading\\JnQ\\'

os.chdir(pkg_path)

# mpl_finance_path = 'D:\\python\\python38_1\\projects\\JnQ\\mpl_finance'
# ta_lib_path = 'D:\\python\\python38_1\\projects\\JnQ\\ta_lib'
funcs_path = pkg_path + 'funcs'

if funcs_path not in sys.path:

  try:
    # sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks/JnQ')
    sys.path.insert(0, pkg_path + 'Bank')
    sys.path.insert(0, funcs_path)
    # sys.path.insert(0, mpl_finance_path)
    # sys.path.insert(0, ta_lib_path)
    
  except Exception as e:
    print(e)

In [2]:
import os
import talib
from funcs.public.idep import *
from funcs.public.plot_check import *
from funcs.public.en_ex_pairing import *
from funcs.public.indicator import *
from funcs.public.broker import *
from funcs.public.ds import *
from ast import literal_eval
import logging
import importlib


import matplotlib.pyplot as plt
from matplotlib import gridspec

# import torch

import numpy as np
import pandas as pd
import scipy.stats as stats
# from sklearn.metrics.pairwise import cosine_similarity

# import bz2
import pickle
# import _pickle as cPickle
import shutil
import json
from easydict import EasyDict
import copy

import datetime
from datetime import datetime
import random
import time
# import warnings

from IPython.display import clear_output
# warnings.simplefilter("ignore", category=RuntimeWarning)

np.seterr(invalid="ignore")
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=2000) 

pd.set_option('mode.chained_assignment',  None)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Telegramer

In [18]:
import telegram
from telegram.ext import Updater
from telegram.ext import MessageHandler, Filters

In [14]:
def echo(self, update, context):
    self.user_text = update.message.text

In [15]:
#       i. Telegram logger
#               1. chat_id 는 env 동일.
token = "6717940201:AAFc62YReeED3mJKw5drHnT7jp6dzXy0rPE"
msg_bot = telegram.Bot(token=token)
chat_id = "5320962614"

#       ii. Telegram messenger
#           1. init.
user_text = None

updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher

echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
dispatcher.add_handler(echo_handler)

#           2. polling for messenger.
updater.start_polling()

In [17]:
msg = "ping telegramer"

try:
    msg_bot.sendMessage(chat_id=chat_id, text=msg)
except Exception as e:
    print("error in msg_bot : {}".format(e))

# Get data

## Upbit

In [26]:
datetime.now().hour#()

23

In [23]:
from funcs.upbit.concat_candlestick_ftr_v2 import concat_candlestick
from pyupbit import get_tickers

"""
1. upbit 에서 당일 기준 이전 데이터를 조회할지라도 None 은 성립하지 않는다. --> end_date = today 로 변경 요구.
"""


days = 1  # 330 3

end_date = None  # None 2023-07-04 # "2023-01-06" "2021-04-12" "2021-03-23"
if end_date is None:
    end_date = str(datetime.now()).split(' ')[0]
    
intervals = [ 'D']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = ['30m', '1h', '2h']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = [ '4h',  '1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old

time_schedule = {'4h': 21, '1d': 9}
    
show_process = False
show_process = True


    
symbol_list = get_tickers()

while 1:

    target_symbol_total = []
    for interval in intervals:

        target_symbol = []
        for s_i, symbol in enumerate(symbol_list):

            print(s_i)

            if 'KRW' not in symbol:
                continue

            try:
                concated_df, end_date = concat_candlestick(symbol, 
                                                           interval, 
                                                           days, 
                                                           limit=limit_by_itv(interval),
                                                           end_date=end_date, 
                                                           show_process=show_process, 
                                                           timesleep=0.1)


                concated_df2 = fisher_v2(concated_df, 30, itv=interval)    
                display(concated_df2.tail())
                break

                target_data = concated_df2.iloc[-2:, -1].to_numpy()

                """
                condition
                """
                # 1. fisher band
                fisher_lower = -1.5
                if target_data[0] < fisher_lower < target_data[1]:
                    target_symbol.append(symbol)

                clear_output(wait=True)          



                # break
                # save_path = os.path.join(save_dir, save_name)
                # concated_df.reset_index().to_feather(save_path, compression='lz4')
                # print(save_path, "saved.\n")
            except Exception as e:
                print("error in save to_feather :", e)
                continue

        target_symbol_total.append([s_.replace('KRW-', '') for s_ in target_symbol])
        print("target_symbol_total :", target_symbol_total)
        # break

0
KRW-BTC
2023-12-21 09:00:00 ~ 2024-01-06 21:00:00


,open,high,low,close,volume,value,fisher_4h30
2024-01-06 05:00:00,58916000.0,59617000.0,58913000.0,59351000.0,503.524362,2.986514e+10,0.428239
2024-01-06 09:00:00,59351000.0,59518000.0,58983000.0,59191000.0,490.309067,2.902600e+10,0.499432
2024-01-06 13:00:00,59191000.0,59200000.0,58651000.0,59014000.0,638.059100,3.758513e+10,0.505456
2024-01-06 17:00:00,59014000.0,59070000.0,58856000.0,58940000.0,310.762314,1.832496e+10,0.495323
2024-01-06 21:00:00,58915000.0,59045000.0,58830000.0,58943000.0,186.809606,1.101254e+10,0.477144


target_symbol_total : [[]]
0
KRW-BTC
2023-09-29 09:00:00 ~ 2024-01-06 09:00:00


,open,high,low,close,volume,value,fisher_1d30
2024-01-02 09:00:00,58839000.0,60997000.0,58814000.0,60206000.0,8221.198684,4.950388e+11,0.792481
2024-01-03 09:00:00,60205000.0,60799000.0,54346000.0,58581000.0,16310.306926,9.458412e+11,0.533018
2024-01-04 09:00:00,58581000.0,60162000.0,57319000.0,59477000.0,6254.537687,3.660256e+11,0.396550
2024-01-05 09:00:00,59477000.0,59695000.0,57577000.0,59351000.0,4822.389677,2.843524e+11,0.347471
2024-01-06 09:00:00,59351000.0,59518000.0,58651000.0,58943000.0,1625.940088,9.594863e+10,0.425548


target_symbol_total : [[], []]


In [7]:
df_res = pd.DataFrame(index=intervals, data=target_symbol_total)
df_res

""
4h
D


In [ ]:
df_res.to_excel("signal/upbit_res.xlsx")

## Binance_Futures

In [183]:
# with open(r"D:\Projects\System_Trading\JnQ\Bank\tickers\binance_20240102.pkl", 'wb') as f:
#     pickle.dump(symbol_list, f)

In [8]:
from binance.um_futures import UMFutures
from funcs.binance.futures_concat_candlestick_ftr_v2 import *
# break


days = 5  # 330 3

end_date = None  # None 2023-07-04 # "2023-01-06" "2021-04-12" "2021-03-23"
if end_date is None:
    end_date = str(datetime.now()).split(' ')[0]
    
intervals = ['4h', '1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = ['15m', '30m', '1h', '4h', '1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old

limit = 1500
    
show_process = False
show_process = True


 
symbol_list = [s_['symbol'] for s_ in um_futures_client.exchange_info()['symbols']]



target_symbol_total = []
for interval in intervals:
    
    target_symbol = []
    for s_i, symbol in enumerate(symbol_list):
        
        print(s_i)
        
        # if 'KRW' not in symbol:
        #     continue
        
        try:
            concated_df, end_date = concat_candlestick_v2(symbol, 
                                                          interval, 
                                                          days,
                                                          limit=limit,
                                                          end_date=None,
                                                          show_process=True,
                                                          timesleep=0.4)
            
            
            concated_df2 = fisher_v2(concated_df, 30, itv=interval.upper())      
            # display(concated_df2.tail())
            # break
            
            target_data = concated_df2.iloc[-2:, -1].to_numpy()
            
            """
            condition
            """
            # 1. fisher band
            fisher_lower = -1.5
            if target_data[0] < fisher_lower < target_data[1]:
                target_symbol.append(symbol)
                
            clear_output(wait=True)          
            
            
            
            # break
            # save_path = os.path.join(save_dir, save_name)
            # concated_df.reset_index().to_feather(save_path, compression='lz4')
            # print(save_path, "saved.\n")
        except Exception as e:
            print("error in save to_feather :", e)
            continue
     
    target_symbol_total.append([s_ + '.P' for s_ in target_symbol])
    # print("target_symbol_total :", target_symbol_total)
    # break

260
BTCUSDC
error in concat_candlestick_v2 : Length mismatch: Expected axis has 0 elements, new values have 5 elements
error in save to_feather : No objects to concatenate
261
ETHUSDC
error in concat_candlestick_v2 : Length mismatch: Expected axis has 0 elements, new values have 5 elements
error in save to_feather : No objects to concatenate
262
BNBUSDC
error in concat_candlestick_v2 : Length mismatch: Expected axis has 0 elements, new values have 5 elements
error in save to_feather : No objects to concatenate
263
SOLUSDC
error in concat_candlestick_v2 : Length mismatch: Expected axis has 0 elements, new values have 5 elements
error in save to_feather : No objects to concatenate
264
XRPUSDC
error in concat_candlestick_v2 : Length mismatch: Expected axis has 0 elements, new values have 5 elements
error in save to_feather : No objects to concatenate


In [9]:
df_res = pd.DataFrame(index=intervals, data=target_symbol_total)
df_res

,0
15m,BLZUSDT.P
30m,None
1h,UNIUSDT.P
4h,None
1d,None


In [ ]:
df_res.to_excel("signal/binance_futures_res.xlsx")